In [2]:
import numpy as np

## padding

In [3]:
array = np.random.rand(3,4)

In [4]:
def padding(array, pad_width, mode, **kwargs):
    if pad_width == 0:
        return array
    if mode == 'constant':
        array = np.pad(array, pad_width, mode, **kwargs)
    return array

In [5]:
def calculate_lenth(array_lenth, kernel_lenth, stride):
    return int((array_lenth - kernel_lenth) / stride + 1)

In [6]:
def convolve(array, kernel, pad:int, stride:int = 1, **kwargs):
    if pad:
        array = padding(array, pad, 'constant', constant_values=0)
        
    kernel_size = kernel.shape
    width = calculate_lenth(array.shape[0], kernel_size[0], stride)
    height = calculate_lenth(array.shape[1], kernel_size[1], stride)
    result = np.zeros(((width, height)))
    
    for i in range(width):
        x = i * stride
        for j in range(height):
            y = j * stride
            if i + kernel_size[0] <= len(array) and j + kernel_size[1] <= len(array[0]):
                result[i, j] = np.dot(array[x:x+kernel_size[0], y:y+kernel_size[1]].flatten(), kernel.flatten())
                # result[i, j] = np.sum(array[x:x+kernel_size[0], y:y+kernel_size[1]] * kernel)
    return result

In [7]:
convolve(array, np.ones((2,2)), pad=1, stride=2)

array([[0.29711984, 0.92347732, 0.83060222],
       [0.58441229, 2.2747577 , 1.33442813]])

In [8]:
def channels_convolve(array, kernel, pad:int, stride:int = 1, channel_first=False, **kwargs):
    if channel_first:
        array = array.transpose(0, 2, 3, 1)
        kernel = kernel.transpose(0, 2, 3, 1)
    result = []
    for i in range(array.shape[0]):
        result.append(convolve(array[i], kernel[i], pad, stride))
    return np.array(result)

In [9]:
def channels_convolve(array, kernel, bias, pad:int = 0, stride:int = 1, channel_last=False, **kwargs):
    if channel_last:
        array = array.transpose(1, 2, 0)
        
    # print(array)
        
    channel = len(array)
    result = []
        
    for i in range(channel):
        result.append(padding(array[i], pad, 'constant', constant_values=0))
        
    array = np.array(result)
    # print(array)
        
    kernel_size = kernel.shape
    output_channel = kernel_size[0]
    width = calculate_lenth(array.shape[1], kernel_size[-2], stride)
    height = calculate_lenth(array.shape[2], kernel_size[-1], stride)
    result = np.zeros(((output_channel, width, height)))
    
    for i in range(width):
        x = i * stride
        for j in range(height):
            y = j * stride
            if i + kernel_size[-2] <= len(array[0]) and j + kernel_size[-1] <= len(array[1]):
                array_comp = array[:, x:x+kernel_size[-2], y:y+kernel_size[-1]].flatten()
                result[:, i, j] = np.dot(array_comp, kernel.flatten().reshape(output_channel, -1).T) + bias
                # result[:, i, j] = np.sum(array[:, x:x+kernel_size[1], y:y+kernel_size[2]] * kernel)
    return result

In [10]:
array = np.random.rand(3,3,4)
kernel = np.ones((3,3,2,2))
bias = np.ones(3)

In [11]:
channels_convolve(array, kernel, bias, pad=1, stride=2, channel_last=False)

array([[[2.40350077, 4.02586307, 1.96670927],
        [3.41423613, 8.16287879, 3.85192792]],

       [[2.40350077, 4.02586307, 1.96670927],
        [3.41423613, 8.16287879, 3.85192792]],

       [[2.40350077, 4.02586307, 1.96670927],
        [3.41423613, 8.16287879, 3.85192792]]])

In [18]:
def batch_channels_convolve(array, kernel, bias, pad=1, stride=2, channel_last=False):
    output = []
    for i in array:
        output.append(channels_convolve(i, kernel, bias, pad, stride, channel_last))
    
    return np.array(output, dtype=np.float32)

In [19]:
batch = 10
array = np.random.rand(batch,3,3,4)
kernel = np.ones((3,3,2,2))
bias = np.ones(3)

In [20]:
batch_channels_convolve(array, kernel, bias)

array([[[[3.0694358, 5.6275134, 1.9081578],
         [3.3432426, 6.947074 , 4.2297325]],

        [[3.0694358, 5.6275134, 1.9081578],
         [3.3432426, 6.947074 , 4.2297325]],

        [[3.0694358, 5.6275134, 1.9081578],
         [3.3432426, 6.947074 , 4.2297325]]],


       [[[2.243345 , 3.7798426, 3.237752 ],
         [3.451369 , 9.062789 , 4.7806826]],

        [[2.243345 , 3.7798426, 3.237752 ],
         [3.451369 , 9.062789 , 4.7806826]],

        [[2.243345 , 3.7798426, 3.237752 ],
         [3.451369 , 9.062789 , 4.7806826]]],


       [[[2.7537405, 2.9384682, 2.653953 ],
         [3.9810932, 7.3085604, 3.7629242]],

        [[2.7537405, 2.9384682, 2.653953 ],
         [3.9810932, 7.3085604, 3.7629242]],

        [[2.7537405, 2.9384682, 2.653953 ],
         [3.9810932, 7.3085604, 3.7629242]]],


       [[[2.0214958, 6.221232 , 2.8142111],
         [2.7397258, 6.702938 , 4.313851 ]],

        [[2.0214958, 6.221232 , 2.8142111],
         [2.7397258, 6.702938 , 4.313851 ]],

    